In [ ]:
from flask import Flask, render_template, request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        search_product = request.form['search-box']
        products = crawl_data(search_product)
        return render_template('result.html', products=products, search_product=search_product)
    return render_template('Donthesitate.html')

def crawl_data(search_product):

    url = "https://m.bunjang.co.kr/"

    driver = webdriver.Chrome()
    driver.get(url)
    driver.fullscreen_window()

    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div/div[2]/div[1]/div[1]/div[1]/div[1]/input'))
    )

    search_box.send_keys(search_product)
    search_box.send_keys(Keys.RETURN)

    product_list = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "sc-cTjmhe"))
    )

    total_price = 0                       
    products = []

    for i in range(1, 101):
        name_xpath = f'//*[@id="root"]/div/div/div[4]/div/div[4]/div/div[{i}]/a/div[2]/div[1]'
        price_xpath = f'//*[@id="root"]/div/div/div[4]/div/div[4]/div/div[{i}]/a/div[2]/div[2]/div[1]'

        try:
            product_name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, name_xpath))
            ).text

            product_price = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, price_xpath))
            ).text

            product_price = int(product_price.replace(',', ''))
            total_price += product_price

            products.append((product_name, product_price))
        except Exception as e:
            print(f'상품 정보를 가져오는 데 실패했습니다. 오류 메시지: {e}')

        if len(products) >= 100:
            break

    driver.quit()

    return products

if __name__ == '__main__':
    app.run(debug=True)